In [55]:
import os
import pandas as pd
import numpy as np
from math import sin, asin, cos, radians, fabs, sqrt
import datetime
import geocoder
import re

In [4]:
# 数据加载
mobike = pd.read_csv('mobike_shanghai_sample_updated.csv')

### 数据评估

#### 质量
* start_time和end_time数据类型不是datetime  

#### 清洁度
* track应该拆分成多个经纬度


In [5]:
mobike.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."


In [6]:
mobike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


In [7]:
# 查看是否有重复数据
mobike.duplicated().value_counts()

False    102361
dtype: int64

### 数据清理

In [8]:
mobike_clean = mobike.copy()

In [9]:
# 去除空值以防error
mobike_clean.dropna(inplace = True) 

#### 质量
* 修改start_time和end_time数据类型为datetime  

In [10]:
# 修改start_time和end_time数据类型为date
mobike_clean['start_time'] = pd.to_datetime(mobike_clean['start_time'])
mobike_clean['end_time'] = pd.to_datetime(mobike_clean['end_time'])

In [11]:
mobike_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null datetime64[ns]
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null datetime64[ns]
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
dtypes: datetime64[ns](2), float64(4), int64(3), object(1)
memory usage: 8.6+ MB


In [12]:
# 添加行程时间 (结束时间 - 开始时间)
mobike_clean['diff_mins'] = mobike_clean['end_time'] - mobike_clean['start_time']
mobike_clean['diff_mins'] = mobike_clean['diff_mins']/np.timedelta64(1,'m')
mobike_clean['diff_mins'] = mobike_clean['diff_mins'].astype(int)

#### 整洁度
* 用split把track以`#`分割开
* 把每个轨迹点再拆分成经纬度，形成新的数据集

In [13]:
# 把轨迹track以#分开
mobike_clean['location'] = mobike_clean['track'].str.split('#')

In [14]:
# 把每个轨迹点拆分成经纬度，形成新的数据集

orderid_list = []
location_x = []
location_y = []

for index in mobike_clean.index:
    for element in mobike_clean['location'].loc[index]:
        x, y = element.split(',')
        orderid_list.append(mobike_clean['orderid'].loc[index])
        location_x.append(x)
        location_y.append(y)
        
track_data = pd.DataFrame({'orderid': orderid_list, 'location_x': location_x, 'location_y': location_y})

In [15]:
# 把经纬度都变成小数类型
track_data.location_x = track_data.location_x.astype(float)
track_data.location_y = track_data.location_y.astype(float)

ValueError: could not convert string to float: '31.29\\'

* 在运行过程中，发现location_y有异常值，不能转换成float  
ValueError: could not convert string to float: '31.29\\\\'

In [16]:
# 查找location_y中带\\的值
track_data.location_y.str.extract('(\d+.\d+\\\\)', expand = False).value_counts()

31.26\    1
31.29\    1
31.28\    1
Name: location_y, dtype: int64

In [17]:
# 去掉\\后的值代替原来的
track_data.location_y = track_data.location_y.replace({'31.29\\': '31.29', '31.26\\': '31.26', '31.28\\': '31.28'})
# 转成小数类型
track_data.location_y = track_data.location_y.astype(float)

In [18]:
track_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381105 entries, 0 to 2381104
Data columns (total 3 columns):
orderid       int64
location_x    float64
location_y    float64
dtypes: float64(2), int64(1)
memory usage: 54.5 MB


In [19]:
track_data.head()

,orderid,location_x,location_y
0,78387,121.347,31.392
1,78387,121.348,31.389
2,78387,121.349,31.390
3,78387,121.350,31.390
4,78387,121.351,31.390


* 根据每个经纬度，算出两点之间的距离(km)  
参考：[python利用地图两个点的经纬度计算两点间距离](https://blog.csdn.net/u013401853/article/details/73368850)

In [20]:
EARTH_RADIUS=6371           # 地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def get_distance_hav(lat0, lng0, lat1, lng1):
    "用haversine公式计算球面两点间的距离。"
    # 经纬度转换成弧度
    lat0 = radians(lat0)
    lat1 = radians(lat1)
    lng0 = radians(lng0)
    lng1 = radians(lng1)
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    distance = 2 * EARTH_RADIUS * asin(sqrt(h))
 
    return '%.2f' % distance

In [21]:
# 计算两点之间的距离
distance = []
for index in track_data.index:
    if index <= 2381103:
        if track_data.orderid[index] != track_data.orderid[index +1]:
            continue
        lon1, lat1 = (track_data.location_x[index], track_data.location_y[index])
        lon2, lat2 = (track_data.location_x[index+1], track_data.location_y[index+1])
        dis = get_distance_hav(lon1,lat1,lon2,lat2)
        distance.append(dis)
    
track_data['distance'] = pd.Series(distance)

In [22]:
# 转成小数类型
track_data['distance'] = track_data['distance'].astype(float)

In [23]:
track_data.head()

,orderid,location_x,location_y,distance
0,78387,121.347,31.392,0.21
1,78387,121.348,31.389,0.13
2,78387,121.349,31.390,0.11
3,78387,121.350,31.390,0.11
4,78387,121.351,31.390,0.06


In [24]:
# 根据每个orderid计算行程的总距离
orderid_dis = track_data['distance'].groupby(track_data['orderid']).sum().to_frame().reset_index()

In [25]:
orderid_dis.head()

,orderid,distance
0,6,1.24
1,24,4.24
2,63,1.94
3,68,4.30
4,72,9.94


In [26]:
orderid_dis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 2 columns):
orderid     102361 non-null int64
distance    102361 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.6 MB


In [27]:
# 设置orderid为index
mobike_clean.set_index('orderid').head()

,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,diff_mins,location
orderid,,,,,,,,,,,
78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",7,"[121.347,31.392, 121.348,31.389, 121.349,31.39..."
891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",22,"[121.489,31.270, 121.489,31.271, 121.490,31.27..."
1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",19,"[121.381,31.251, 121.382,31.251, 121.382,31.25..."
1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",9,"[121.471,31.325, 121.472,31.325, 121.473,31.32..."
188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",9,"[121.407,31.291, 121.407,31.292, 121.408,31.29..."


In [28]:
# 根据orderid，合并总距离到原数据中
mobike_clean_com = mobike_clean.join(orderid_dis.set_index('orderid'), on = 'orderid')

In [29]:
# 删除track列
mobike_clean_com = mobike_clean_com.drop('track', axis = 1)

In [30]:
# 按起始日期计算星期，以便之后区分工作日和周末
weekday = []

for i in mobike_clean_com['start_time']:
    weekday.append(i.weekday())
    
mobike_clean_com['weekday'] = pd.Series(weekday)

In [31]:
mobike_clean_com.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,diff_mins,location,distance,weekday
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,7,"[121.347,31.392, 121.348,31.389, 121.349,31.39...",1.94,5
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,22,"[121.489,31.270, 121.489,31.271, 121.490,31.27...",3.36,0
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,19,"[121.381,31.251, 121.382,31.251, 121.382,31.25...",4.58,5
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,9,"[121.471,31.325, 121.472,31.325, 121.473,31.32...",2.54,1
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,9,"[121.407,31.291, 121.407,31.292, 121.408,31.29...",1.02,1


##### 备注
* weekday表示一周中的第几天，数字0-6代表周一至周日

In [32]:
mobike_clean_com.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,diff_mins,location,distance,weekday
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,7,"[121.347,31.392, 121.348,31.389, 121.349,31.39...",1.94,5
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,22,"[121.489,31.270, 121.489,31.271, 121.490,31.27...",3.36,0
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,19,"[121.381,31.251, 121.382,31.251, 121.382,31.25...",4.58,5
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,9,"[121.471,31.325, 121.472,31.325, 121.473,31.32...",2.54,1
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,9,"[121.407,31.291, 121.407,31.292, 121.408,31.29...",1.02,1


### 数据导出

In [33]:
# 数据导出
track_data.to_csv('track_data.csv')

In [34]:
mobike_clean_com.to_csv('mobike_clean_data.csv', index = False)